In [38]:
source(file = "R/packages.R")
source(file = "R/multiplots.R")
source(file = "R/variables.R")
source(file = "R/utils.R")
source(file = "R/plots.R")

library(ggplot2)
library(xtable)
library(ellipse)
library(reshape)
library(digest)
library(scales)
library(cliffsd)

source(file = "R/merge-time.R")

library(doMC)
registerDoMC(num.processes)

source(file = "R/packages.R")
source(file = "R/utils.R")
source(file = "R/variables.R")

library(ggplot2)

Warning message in file(filename, "r", encoding = encoding):
“cannot open file 'R/packages.R': No such file or directory”

ERROR: Error in file(filename, "r", encoding = encoding): cannot open the connection


In [2]:
# Loading data files
all <- load.data()

[1] "Reading file data/A24Group@ssGWT-lib.csv"
[1] "Reading file data/aeolusproject@conductor.csv"
[1] "Reading file data/aeolusproject@imagefactory.csv"
[1] "Reading file data/akka@akka.csv"
[1] "Reading file data/alibaba@druid.csv"
[1] "Reading file data/allegro@ralph.csv"
[1] "Reading file data/alphagov@frontend.csv"
[1] "Reading file data/alphagov@smart-answers.csv"
[1] "Reading file data/alphagov@whitehall.csv"
[1] "Reading file data/AppScale@appscale-tools.csv"
[1] "Reading file data/AppScale@appscale.csv"
[1] "Reading file data/asciidoctor@asciidoctor.csv"
[1] "Reading file data/atlasapi@atlas.csv"
[1] "Reading file data/autotest@autotest.csv"
[1] "Reading file data/autotest@virt-test.csv"
[1] "Reading file data/avoinministerio@avoinministerio.csv"
[1] "Reading file data/BaseXdb@basex.csv"
[1] "Reading file data/bbatsov@rubocop.csv"
[1] "Reading file data/bigbluebutton@bigbluebutton.csv"
[1] "Reading file data/Bitmessage@PyBitmessage.csv"
[1] "Reading file data/boto@boto.csv"
[1

In [3]:
run.mergetime.classifiers <- function(df, cases = c(1000, 10000,
                                                    floor(nrow(df)/4),
                                                    floor(nrow(df)/2),
                                                    nrow(df)),
                                      bins = 3, suffix = "") {
  splitter <- get(sprintf("prepare.data.mergetime.%dbins", bins))
  for (i in cases) {
    cvResult <- cross.validation(merge.time.model,
                                 run.classifiers.mergetime,
                                 splitter, df, i, 10)
    write.csv(cvResult, file = sprintf("merge-time-cv-%dbins-%d%s.csv", bins, i, suffix))
    cross.validation.plot(cvResult,
                          sprintf("Merge time task cross validation (%d classes, %d items)",bins, i),
                          sprintf("merge-time-cv-%dbins-%d%s.pdf",bins, i, suffix))
  }
}

In [5]:
run.mergetime.classifiers(all)

ERROR: Error in {: task 1 failed - "undefined columns selected"


# Random Forest Analysis

In [15]:
setwd('/Volumes/Second_HD/GIT_DIR_2/cs846-Project/pullreqs/results/')

In [18]:
list.files()

[1] "merge-decision-cv-1000-dominant.csv"    
 [2] "merge-decision-cv-1000.csv"             
 [3] "merge-decision-cv-10000-dominant.csv"   
 [4] "merge-decision-cv-10000.csv"            
 [5] "merge-decision-cv-166884-dominant.csv"  
 [6] "merge-decision-cv-166884.csv"           
 [7] "merge-decision-cv-41721-dominant.csv"   
 [8] "merge-decision-cv-41721.csv"            
 [9] "merge-decision-cv-83442-dominant.csv"   
[10] "merge-decision-cv-83442.csv"            
[11] "merge-time-cv-3bins-1000-dominant.csv"  
[12] "merge-time-cv-3bins-1000.csv"           
[13] "merge-time-cv-3bins-10000-dominant.csv" 
[14] "merge-time-cv-3bins-10000.csv"          
[15] "merge-time-cv-3bins-166884-dominant.csv"
[16] "merge-time-cv-3bins-166884.csv"         
[17] "merge-time-cv-3bins-41721-dominant.csv" 
[18] "merge-time-cv-3bins-41721.csv"          
[19] "merge-time-cv-3bins-83442-dominant.csv" 
[20] "merge-time-cv-3bins-83442.csv"

In [21]:
# Load data
merge.time.data = read.csv('merge-time-cv-3bins-10000.csv')
merge.decision.data = read.csv('merge-decision-cv-10000.csv')

In [27]:
aggregate(auc ~ classifier, merge.time.data, mean)
aggregate(acc ~ classifier, merge.time.data, mean)
# aggregate(prec ~ classifier, merge.time.data, mean)
# aggregate(rec ~ classifier, merge.time.data, mean)

aggregate(auc ~ classifier, merge.decision.data, mean)
aggregate(acc ~ classifier, merge.decision.data, mean)
aggregate(prec ~ classifier, merge.decision.data, mean)
aggregate(rec ~ classifier, merge.decision.data, mean)

rf.merge.decision <- subset(merge.decision.data, classifier == "randomforest")
rf.merge.time <- subset(merge.time.data, classifier == "randomforest")

print(sprintf("std dev auc merge time: %f", sd(rf.merge.time$auc)))
print(sprintf("std dev auc merge decision: %f", sd(rf.merge.decision$auc)))


classifier,auc
multinomregr,0.6041746
naivebayes,0.6045404
randomforest,0.6755963


classifier,acc
multinomregr,0.4399074
naivebayes,0.3904068
randomforest,0.5086486


classifier,auc
binlogregr,0.7563407
naive bayes,0.7140626
randomforest,0.9036435


classifier,acc
binlogregr,0.59600
naive bayes,0.58465
randomforest,0.85090


classifier,prec
binlogregr,0.9507320
naive bayes,0.9375079
randomforest,0.9546679


classifier,rec
binlogregr,0.5563607
naive bayes,0.5501206
randomforest,0.8698603


[1] "std dev auc merge time: 0.013415"
[1] "std dev auc merge decision: 0.012578"


# Variable Importance

In [22]:
setwd('/Volumes/Second_HD/GIT_DIR_2/cs846-Project/pullreqs/')
source(file = "R/variables.R")
source(file = "R/utils.R")
source(file = "R/classification.R")
source(file = "R/merge-time.R")
source(file = "R/merge-decision.R")

library(ggplot2)
library(doMC)
registerDoMC(num.processes)

In [6]:
library(sqldf)

In [24]:
merge.decision.model <- merged ~ team_size + num_commits + files_changed +
  perc_external_contribs + sloc + src_churn + test_churn + num_comments +
  commits_on_files_touched +  test_lines_per_kloc + prev_pullreqs +
  requester_succ_rate + main_team_member + conflict + forward_links

In [14]:
# Strip a project data frame from unused columns
prepare.project.df <- function(a) {
  a[,c(7:32)]
}


In [15]:
# Returns a list l where 
# l[1] training dataset
# l[2] testing dataset
prepare.data.mergedecision <- function(df, num_samples) {
  # Prepare the data for prediction
  a <- prepare.project.df(df)

  if (num_samples >= nrow(a)) {
    num_samples = nrow(a) - 1
  }

  # Take sample
  a <- a[sample(nrow(a), size=num_samples), ]

  # Remove column mergetime_minutes as it contains NAs
  a <- a[-c(1)]

  # split data into training and test data
  a.train <- a[1:floor(nrow(a)*.90), ]
  a.test <- a[(floor(nrow(a)*.90)+1):nrow(a), ]
  list(train=a.train, test=a.test)
}

In [16]:
rf.varimp <- function(model, sampler, data, num_samples = 5000, runs = 50) {

  result <- foreach(n=1:runs, .combine=rbind) %dopar% {
      df <- sampler(data, num_samples)
      rfmodel <- randomForest(model, data=df$train, importance = T,
                              type = "classification", mtry = 5,
                              ntree = 2000)
      print(importance(rfmodel))
      i <- data.frame(importance(rfmodel))
      i$var <- row.names(i)
      i$var <- as.factor(i$var)
      i$run <- n
      i
  }

  result = aggregate(. ~ var, data = result, mean)
  result = result[with(result, order(-MeanDecreaseAccuracy)),]
  result[c('var', 'MeanDecreaseAccuracy')]
}

In [17]:
run.rf.varimp <- function(expname, model, sampler, data, smpl_size, runs) {
  varimp <- rf.varimp(model, sampler, data, smpl_size, runs)
  name.for.file <- gsub(" ", "-", tolower(expname))
  chart.title = sprintf("%s variable importance\n(n = %d, ntree = 2000, mtry = 5, runs = %d)",
                        expname, smpl_size, runs)

  p <- ggplot(varimp, aes(y = reorder(MeanDecreaseAccuracy, var), x = MeanDecreaseAccuracy)) +
    geom_point(size = 3) +
    scale_y_discrete("Variable", label = rev(varimp$var)) +
    xlab("Mean Decrease in Accuracy") +
    ggtitle(chart.title)

  store.pdf(p, plot.location, sprintf("varimp-%s-%d-%d.pdf", name.for.file, smpl_size, runs))
  print(varimp)
}

In [21]:
run.rf.varimp("Merge decision", merge.decision.model, prepare.data.mergedecision, all, floor(nrow(all)/2), 50)
run.rf.varimp("Merge time (3 classes)", merge.time.model, prepare.data.mergetime.3bins, all, floor(nrow(all)/2), 50)


ERROR: Error in {: task 1 failed - "object 'num_participants' not found"


In [18]:
expname<-"Merge decision"
model<-merge.decision.model
sampler<-prepare.data.mergedecision
data<-all
smpl_size<-floor(nrow(all)/2)
runs<-50

In [25]:
varimp <- rf.varimp(model, sampler, data, smpl_size, runs)

ERROR: Error in {: task 1 failed - "object 'num_participants' not found"


In [47]:
varimp <- rf.varimp(model, sampler, data, smpl_size, runs)
name.for.file <- gsub(" ", "-", tolower(expname))
chart.title = sprintf("%s variable importance\n(n = %d, ntree = 2000, mtry = 5, runs = %d)",
                    expname, smpl_size, runs)

ERROR: Error in {: task 1 failed - "could not find function "sampler""


In [20]:
all

,pull_req_id,project_name,lang,github_id,created_at,merged_at,closed_at,lifetime_minutes,mergetime_minutes,merged_using,⋯,test_cases_per_kloc,asserts_per_kloc,watchers,requester,prev_pullreqs,requester_succ_rate,followers,intra_branch,main_team_member,merged
1,1998470,A24Group/ssGWT-lib,java,220,1377693657,1377748331,1377748331,911,911,github,⋯,2.208983,4.953477,5,nauderuan,25,0.9600000,5,TRUE,FALSE,TRUE
2,1904236,A24Group/ssGWT-lib,java,219,1375887412,1375937076,1375937076,827,827,github,⋯,2.209575,4.954804,4,Rynoh,11,1.0000000,6,TRUE,FALSE,TRUE
3,1857406,A24Group/ssGWT-lib,java,218,1375078176,1375106234,1375106235,467,467,github,⋯,2.251638,5.049127,4,michaelbarnard,42,0.9285714,10,FALSE,FALSE,TRUE
4,1807435,A24Group/ssGWT-lib,java,217,1374499015,1374499278,1374499279,4,4,github,⋯,2.251638,5.049127,4,Sh33pman,35,1.0000000,8,TRUE,FALSE,TRUE
5,1805341,A24Group/ssGWT-lib,java,216,1374475132,1375344073,1375344074,14482,14482,github,⋯,2.251638,5.049127,4,nauderuan,24,0.9583333,5,TRUE,FALSE,TRUE
6,2006608,A24Group/ssGWT-lib,java,215,1374239632,1374240401,1374240401,12,12,github,⋯,2.251791,5.049471,4,Sh33pman,34,1.0000000,8,TRUE,FALSE,TRUE
7,1790115,A24Group/ssGWT-lib,java,214,1374143769,1374144839,1374144839,17,17,github,⋯,2.757123,6.182638,4,michaelbarnard,41,0.9268293,10,FALSE,FALSE,TRUE
8,1784253,A24Group/ssGWT-lib,java,213,1374052641,1374052692,1374052692,0,0,github,⋯,2.757123,6.182638,4,NeilNienaber,3,1.0000000,0,TRUE,FALSE,TRUE
9,1784226,A24Group/ssGWT-lib,java,212,1374052182,1374052517,1374052517,5,5,github,⋯,2.757814,6.184189,4,NeilNienaber,2,1.0000000,0,TRUE,FALSE,TRUE
10,1773012,A24Group/ssGWT-lib,java,210,1373886855,1373899963,1373899963,218,218,github,⋯,2.757814,6.184189,4,AlecErasmus,56,0.9642857,3,TRUE,FALSE,TRUE
